In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
train_dir = './train_test_split/train/'
test_dir = './train_test_split/test/'
val_dir = './train_test_split/valid/'


image_size = (224, 224)
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

Found 2576 images belonging to 2 classes.
Found 240 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [3]:
NUM_CLASSES = 2

# Fixed for Cats & Dogs color images
CHANNELS = 3

IMAGE_RESIZE = 256
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'binary_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 100
EARLY_STOP_PATIENCE = 15

STEPS_PER_EPOCH_TRAINING = 24
STEPS_PER_EPOCH_VALIDATION = 12

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Model

# Define the input size of your EEG spectrogram images
input_shape = (224, 224, 3)  # Adjust the size as needed

# Load the pre-trained ResNet-152 model without the top classification layer
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers (optional, you can experiment with unfreezing)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling layer
x = Dense(128, activation='relu')(x)  # Add a fully connected layer (adjust the number of units)
predictions = Dense(2, activation='softmax')(x)  # Output layer for binary classification

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary to view the architecture
model.summary()


234698864/234698864 [==============================] - 26s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                      

 conv2_block2_out (Activati  (None, 56, 56, 256)          0         ['conv2_block2_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 56, 56, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 56, 56, 64)           256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, 56, 56, 64)           0         ['conv2_block3_1_bn[0][0]']   
 ation)   

                                                                                                  
 conv3_block2_2_relu (Activ  (None, 28, 28, 128)          0         ['conv3_block2_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 28, 28, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, 28, 28, 512)          2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_blo

 ation)                                                                                           
                                                                                                  
 conv3_block5_2_conv (Conv2  (None, 28, 28, 128)          147584    ['conv3_block5_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block5_2_bn (BatchNo  (None, 28, 28, 128)          512       ['conv3_block5_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block5_2_relu (Activ  (None, 28, 28, 128)          0         ['conv3_block5_2_bn[0][0]']   
 ation)                                                                                           
          

 conv3_block8_1_conv (Conv2  (None, 28, 28, 128)          65664     ['conv3_block7_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv3_block8_1_bn (BatchNo  (None, 28, 28, 128)          512       ['conv3_block8_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block8_1_relu (Activ  (None, 28, 28, 128)          0         ['conv3_block8_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv3_block8_2_conv (Conv2  (None, 28, 28, 128)          147584    ['conv3_block8_1_relu[0][0]'] 
 D)       

...

                                                                                                  
 conv4_block27_2_relu (Acti  (None, 14, 14, 256)          0         ['conv4_block27_2_bn[0][0]']  
 vation)                                                                                          
                                                                                                  
 conv4_block27_3_conv (Conv  (None, 14, 14, 1024)         263168    ['conv4_block27_2_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv4_block27_3_bn (BatchN  (None, 14, 14, 1024)         4096      ['conv4_block27_3_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_blo

 vation)                                                                                          
                                                                                                  
 conv4_block30_2_conv (Conv  (None, 14, 14, 256)          590080    ['conv4_block30_1_relu[0][0]']
 2D)                                                                                              
                                                                                                  
 conv4_block30_2_bn (BatchN  (None, 14, 14, 256)          1024      ['conv4_block30_2_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_block30_2_relu (Acti  (None, 14, 14, 256)          0         ['conv4_block30_2_bn[0][0]']  
 vation)                                                                                          
          

 conv4_block33_1_conv (Conv  (None, 14, 14, 256)          262400    ['conv4_block32_out[0][0]']   
 2D)                                                                                              
                                                                                                  
 conv4_block33_1_bn (BatchN  (None, 14, 14, 256)          1024      ['conv4_block33_1_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_block33_1_relu (Acti  (None, 14, 14, 256)          0         ['conv4_block33_1_bn[0][0]']  
 vation)                                                                                          
                                                                                                  
 conv4_block33_2_conv (Conv  (None, 14, 14, 256)          590080    ['conv4_block33_1_relu[0][0]']
 2D)      

                                                                                                  
 conv4_block35_add (Add)     (None, 14, 14, 1024)         0         ['conv4_block34_out[0][0]',   
                                                                     'conv4_block35_3_bn[0][0]']  
                                                                                                  
 conv4_block35_out (Activat  (None, 14, 14, 1024)         0         ['conv4_block35_add[0][0]']   
 ion)                                                                                             
                                                                                                  
 conv4_block36_1_conv (Conv  (None, 14, 14, 256)          262400    ['conv4_block35_out[0][0]']   
 2D)                                                                                              
                                                                                                  
 conv4_blo

 D)                                                                                               
                                                                                                  
 conv5_block2_2_bn (BatchNo  (None, 7, 7, 512)            2048      ['conv5_block2_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block2_2_relu (Activ  (None, 7, 7, 512)            0         ['conv5_block2_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block2_3_conv (Conv2  (None, 7, 7, 2048)           1050624   ['conv5_block2_2_relu[0][0]'] 
 D)                                                                                               
          

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = r'./ResNet-152.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')


fit_history = model.fit(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        validation_data=val_generator,
        validation_steps=STEPS_PER_EPOCH_VALIDATION,
        callbacks=[cb_checkpointer, cb_early_stopper]
)

Epoch 1/100
24/24 [==============================] - ETA: 0s - loss: 0.7366 - accuracy: 0.4661

C:\Users\admin\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - 119s 4s/step - loss: 0.7366 - accuracy: 0.4661 - val_loss: 0.7334 - val_accuracy: 0.4844
Epoch 2/100
24/24 [==============================] - 96s 4s/step - loss: 0.7150 - accuracy: 0.4531 - val_loss: 0.6928 - val_accuracy: 0.5208
Epoch 3/100
24/24 [==============================] - 94s 4s/step - loss: 0.7157 - accuracy: 0.4661 - val_loss: 0.7274 - val_accuracy: 0.5000
Epoch 4/100
24/24 [==============================] - 95s 4s/step - loss: 0.7148 - accuracy: 0.4948 - val_loss: 0.6973 - val_accuracy: 0.4948
Epoch 5/100
24/24 [==============================] - 99s 4s/step - loss: 0.7017 - accuracy: 0.4583 - val_loss: 0.6924 - val_accuracy: 0.5312
Epoch 6/100
24/24 [==============================] - 96s 4s/step - loss: 0.6991 - accuracy: 0.4896 - val_loss: 0.7478 - val_accuracy: 0.4948
Epoch 7/100
24/24 [==============================] - 96s 4s/step - loss: 0.7133 - accuracy: 0.5026 - val_loss: 0.7033 - val_accuracy: 0.4948
Epoch 8/100
24/24 [=====